# Week 8

Patrick Weatherford

***


<br>

## Import Modules

Mount Google Drive to Colab session

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


<br>Copy custom packages/modules over to default path where python packages are installed for this Colab session.

Link to custom modules on GitHub:


In [2]:
import os
import site
import shutil

g_drive_mod_path = r"/content/drive/MyDrive/Bellevue_University/Python/Custom_Modules"

# create custom modules folder in default pkg location
colab_mod_path = site.getsitepackages()[0]  # get path where packages are installed
colab_cust_mod_path = f"{colab_mod_path}/hakuna_patata_modules"

if os.path.exists(colab_cust_mod_path):
    print('Directory not copied! Directory already exists!')
else:
    try:
        shutil.copytree(g_drive_mod_path, colab_cust_mod_path)
        print(f"{os.path.basename(g_drive_mod_path)} successfully copied from {g_drive_mod_path} to {colab_cust_mod_path}")
    except:
        print(f"Copy failed!")


Custom_Modules successfully copied from /content/drive/MyDrive/Bellevue_University/Python/Custom_Modules to /usr/local/lib/python3.7/dist-packages/hakuna_patata_modules


In [3]:
from hakuna_patata_modules import (
    api_keys,
    hp_kaggle,
    sklearn_df
)
from zipfile import ZipFile
import pandas as pd 
from pandas import DataFrame as DF
import numpy as np

from sklearn.pipeline import Pipeline

from google.colab.data_table import DataTable
def DT(df, num_rows_per_page=10):
    return DataTable(df, num_rows_per_page=num_rows_per_page)


***

<br>

## Import Data

In [4]:
kaggle_api = hp_kaggle.kaggle_api(api_keys.Kaggle_API['username'], api_keys.Kaggle_API['key'])


In [5]:
hp_kaggle.kaggle_dataset_download(
    kaggle_api,
    r"granjithkumar/loan-approval-data-set",
    ['Loan_Train.csv'],
    r"/content/kaggle_datasets"
)


Loan_Train.csv successfully downloaded to /content/kaggle_datasets


In [6]:
origin_df = pd.read_csv("kaggle_datasets/Loan_Train.csv")

DT(origin_df)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


***

<br>

## Create Transformer Pipeline

In [19]:

transform_pipe = Pipeline([
    ('drop_cols', sklearn_df.DFColumnDropper(['Loan_ID'])),
    ('drop_nan_rows', )
])

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [13]:
class DFDropNaN():
    def fit(self, X, y=None):
        return self 

    def transform(self, X, *args, **kwargs):
        return X.dropna(*args, **kwargs)


new_df = DFDropAllNaN().transform(origin_df, how='any')

new_df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [29]:
keep_cols = [col for col in origin_df.columns if col not in ['Loan_ID']]